# Import Package

In [1]:
import pandas as pd
import numpy as np
import os
from math import floor
from operator import add

In [2]:
from google.colab import drive
drive.mount('/content/drive')

volatility_folder = "/content/drive/MyDrive/fyp/Final report/4. Backtesting/Volatility_result/"
backtest_summary_folder = volatility_folder + "backtest_summary/"
record_folder = volatility_folder + "trade_record/"
user_test_folder = volatility_folder + "user_test/"

Mounted at /content/drive


# Calculate model-stock volatility based on the year

In [ ]:

def get_sd_file(start_year,end_year):
  summary_to_process = list(filter(lambda file: "_sd" not in file and "_mean" not in file,os.listdir(backtest_summary_folder)))
  for file in summary_to_process:
    df = pd.read_csv(backtest_summary_folder+file,index_col = 0)
    df = df[(df["Range_Start"] >= start_year*10000+101) & (df["Range_Start"] <= end_year*10000+101)]
    new_sd_summary = df.groupby(["Sector","Stock"])[["Sector",
                                                    "Stock",
                                                    "CAGR_Performance(in %)",
                                                    "MDD_Performance(in %)"]].std()
                                                    
    new_sd_summary = new_sd_summary.reset_index()
    new_sd_summary.dropna(inplace = True)
    new_sd_summary.to_csv(backtest_summary_folder+file[:-4]+"_sd.csv")
    print("Exported... for",file)

get_sd_file(2012,2018)

Exported... for [5]backtest_summary_SVC.csv
Exported... for [5]backtest_summary_SVR.csv
Exported... for backtest_summary_twap.csv
Exported... for backtest_summary_CNN_LSTM_CLASS.csv
Exported... for backtest_summary_CNN_LSTM_REG.csv
Exported... for backtest_summary_CAT.csv
Exported... for backtest_summary_Xgb.csv


# Make Volatility Summary

In [4]:

model_bactest_summary_list = {"[5]SVC":"[5]backtest_summary_SVC_sd.csv",
                              "[5]SVR":"[5]backtest_summary_SVR_sd.csv",
                              "CNN_LSTM_CLASS":"backtest_summary_CNN_LSTM_CLASS_sd.csv",
                              "CNN_LSTM_REG":"backtest_summary_CNN_LSTM_REG_sd.csv",
                              "CAT":"backtest_summary_CAT_sd.csv",
                              "XGB":"backtest_summary_Xgb_sd.csv",
                              "TWAP":"backtest_summary_twap_sd.csv"}

volatility_summary = pd.DataFrame(columns=["Sector","Stock","Model","Volatility"])

for model,file in model_bactest_summary_list.items():
    df = pd.read_csv(backtest_summary_folder+file,index_col = 0) # Read the model-stock volatility
    df = df.rename(columns={"CAGR_Performance(in %)":"Volatility"}) # Use CAGR Performance SD as volatility
    df = df[["Sector","Stock","Volatility"]] # Only Keep necessary columns
    df["Model"] = model # Record which model
    
    # Reorder the col
    volatility_summary = volatility_summary.append(df,ignore_index = True)
    
    # Standardize sector format
    volatility_summary["Sector"] = list(map(lambda sector: sector.title(), volatility_summary["Sector"]))

volatility_summary = volatility_summary.sort_values(by=["Volatility"],ascending = [False])
volatility_summary.to_csv(volatility_folder+"Stock_Model_Volatility_summary.csv",index = False)   
print("Export Completed...")

Export Completed...


# Statistics on Volatility Summary

In [ ]:
print("VolatilityMax:",max(volatility_summary["Volatility"]))
print("VolatilityMin:",min(volatility_summary["Volatility"]))
print("VolatilityMean:",np.mean(volatility_summary["Volatility"]))


VolatilityMax: 153.05928468593936
VolatilityMin: 0.9475230867899735
VolatilityMean: 24.28400589176513


# Set cut off and show counting

In [5]:
# Below cutoff_1 => Low Risk Level
# Between cutoff_1 and cutoff_2 => Medium Risk Level
# above cutoff_2 => High Risk Level
cutoff_1 = 13
cutoff_2 = 30

low = volatility_summary[(volatility_summary["Volatility"] <= cutoff_1)]["Volatility"].count() 
medium = volatility_summary[(volatility_summary["Volatility"] > cutoff_1) & (volatility_summary["Volatility"] <= cutoff_2)]["Volatility"].count() 
high = volatility_summary[volatility_summary["Volatility"] > cutoff_2]["Volatility"].count() 

# Count of stock in each level
print("Low Level:\t",low)
print("Medium Level:\t", medium)
print("High Level\t", high)



Low Level:	 83
Medium Level:	 157
High Level	 75


# Assign risk_level

In [6]:
# Append the risk level column

def classify_risk_level(volatility):
    global cutoff_1
    global cutoff_2
    if volatility <= cutoff_1:
        return "low"
    elif volatility <= cutoff_2:
        return "medium"
    else:
        return "high"
    
volatility_summary["risk_level"] = volatility_summary["Volatility"].apply(classify_risk_level)

In [7]:
volatility_summary

,Sector,Stock,Model,Volatility,risk_level
300,Semiconductors,AMD,TWAP,153.059285,high
75,Semiconductors,AMD,[5]SVR,115.584027,high
210,Semiconductors,AMD,CAT,107.803002,high
120,Semiconductors,AMD,CNN_LSTM_CLASS,106.605977,high
192,Entertainment,NFLX,CAT,102.724870,high
...,...,...,...,...,...
136,Airline,GD,CNN_LSTM_REG,5.337012,low
144,Energy & Resources,DD,CNN_LSTM_REG,4.644089,low
175,Telecommunication,T,CNN_LSTM_REG,4.365728,low
159,Real Estate,SPG,CNN_LSTM_REG,3.556537,low


# Get dummy user

In [8]:
user_df = pd.read_csv(volatility_folder + "user.csv",index_col = 0)

def get_risk_level_by_id(user_df, uid):
  return user_df.loc[uid,"risk_level"]

In [9]:
user_df

,age,portion,item,risk,why,term,bearing,types,profit_exp,conf,res_1,res_2,tf1,tf2,tf3,tf4,stock_type,risk_level
1,31-45,0-20%,20-40k,10-20%,2. Only source of profit,Short Term (within 3 months),1,['ETF'],>80%,7,2. Sell majority of stock and take some time b...,1. Sell everything and gain profit,Long term,True,False,True,"['Telecommunication', 'Franchise', 'Traveling'...",low
2,18-22,81-100%,0-10k,10-20%,1. Stock as a part of portfolio,Regular (6month~1 year),1,"['Leverage', 'ETF']",40-80%,9,5. Remain calm and keep the portfolio as is,"3. Take out 50%, and wait for further growth",Short term,False,Don't know,Don't know,"['Semiconductors', 'Traveling']",medium
3,31-45,0-20%,20-40k,10-20%,3. As a part of experience :),Short Term (within 3 months),6,"['Fund', 'ETF', 'Individual Stocks']",10-20%,3,1. Sell everything and give up,"4. Volatility is inevitable, so only sell the ...",Don't know,False,False,Don't know,"['Entertainment', 'Energy & Resources', 'Trave...",low
4,18-22,81-100%,10-20k,10-20%,3. As a part of experience :),Short Term (within 3 months),2,"['Fund', 'Individual Stocks', 'Leverage', 'ETF']",20-40%,6,2. Sell majority of stock and take some time b...,5. Keep the portfolio as is,Long term,Don't know,False,True,"['Cloud Computing', 'BlockChain', 'Real Estate...",medium
5,31-45,61-80%,10-20k,over 30%,3. As a part of experience :),Long (More than 3 years),6,['Fund'],>80%,8,"3. Take out 50%, and wait before the rebound","3. Take out 50%, and wait for further growth",Short term,False,False,False,"['Luxury goods', 'Semiconductors', 'Airline', ...",medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,31-45,81-100%,10-20k,over 30%,2. Only source of profit,Regular (6month~1 year),1,['Fund'],20-40%,10,5. Remain calm and keep the portfolio as is,5. Keep the portfolio as is,Don't know,False,True,Don't know,"['Semiconductors', 'Traveling', 'Airline']",medium
997,46-65,21-40%,40-80k,10-20%,1. Stock as a part of portfolio,Quite Long (1~3 year),9,"['Fund', 'Individual Stocks', 'Leverage', 'ETF']",10-20%,8,"4. Volatility is inevitable, so only sell the ...",1. Sell everything and gain profit,Don't know,False,True,False,"['Social Media', 'Luxury goods', 'Franchise', ...",medium
998,>65,81-100%,20-40k,10-20%,3. As a part of experience :),Regular (6month~1 year),10,"['Fund', 'Leverage', 'ETF']",0-10%,7,2. Sell majority of stock and take some time b...,1. Sell everything and gain profit,Short term,True,True,Don't know,"['Entertainment', 'Retail', 'Airline', 'Semico...",high
999,23-30,81-100%,20-40k,over 30%,2. Only source of profit,Short Term (within 3 months),9,"['Individual Stocks', 'Fund', 'Leverage']",>80%,2,"4. Volatility is inevitable, so only sell the ...",1. Sell everything and gain profit,Short term,True,False,True,['Luxury goods'],high


# Get the top X stock-model list by User ID

In [10]:
def get_user_sector_stock_by_id(user_df,volatility_summary,uid,top = -1):
    uid_want = user_df[user_df.index == uid]
    uid_want_stock = uid_want.loc[uid,"stock_type"]
    uid_risk_lv = uid_want.loc[uid,"risk_level"]

    sector_list = list(uid_want_stock[1:-1].split(","))
    sector_list = list(map(lambda sector: sector.replace("'",""),sector_list))
    sector_list = list(map(lambda sector: sector.title(),sector_list))
    sector_list = list(map(lambda sector: sector.strip(),sector_list))

    volatility_okay = volatility_summary.loc[(volatility_summary["risk_level"]==uid_risk_lv) & (volatility_summary["Sector"].isin(sector_list))]
    volatility_okay = volatility_okay.sort_values(by=["Volatility"],ascending=[False])
    volatility_okay = volatility_okay.reset_index(drop=True)
    if top > -1:
        volatility_okay = volatility_okay.head(top)
    return volatility_okay






In [12]:
get_user_sector_stock_by_id(user_df,volatility_summary,13)

,Sector,Stock,Model,Volatility,risk_level
0,Telecommunication,T,TWAP,12.952671,low
1,Luxury Goods,EL,CNN_LSTM_REG,12.746988,low
2,Telecommunication,T,CAT,12.556685,low
3,Telecommunication,VZ,[5]SVR,12.384909,low
4,Telecommunication,CHTR,CNN_LSTM_REG,12.348033,low
5,Luxury Goods,EL,CNN_LSTM_CLASS,12.282841,low
6,Telecommunication,CMCSA,XGB,12.140834,low
7,Airline,GD,CNN_LSTM_CLASS,11.880257,low
8,Telecommunication,VZ,CAT,11.706905,low
9,Franchise,YUM,CAT,11.386209,low


# User Portfolio Setting

In [ ]:
max_num_stock = 3
user_id = 1

# Set up for the column naming
total_cash_col = []
total_value_col =[]
total_cum_profit = []
for i in range(max_num_stock):
    total_cash_col.append("Cash_Bal_"+str(i+1))
    total_value_col.append("Value_"+str(i+1))
    total_cum_profit.append("Cum_Profit_"+str(i+1))

In [ ]:
# Function to use

def sector_sync(ori_sector):
    sector_speacial_handle = {"Blockchain":"BlockChain","Luxury Goods":"Luxury goods"}
    for k,v in sector_speacial_handle.items():
        if ori_sector == k:
            return v
    return ori_sector

def stock_sync(ori_stock):
    sector_speacial_handle = {}
    for k,v in sector_speacial_handle.items():
        if ori_stock == k:
            return v
    return ori_stock



def get_buy_and_hold_result(summary,capital = 10000,handle_fee = 0.0005):
    num_stock = sum(list(map(lambda col: "Price_" in col,summary.columns)))
    dp = 3
    #print(num_stock)
    for i in range(num_stock):
        #print("***",i)
        balance = capital
        price_series = summary["Price_"+str(i+1)]
        #print(price_series)
        bnh_series = []
        total_capital = []
        
        # Get the first price (if not found then go to next day)
        j = 0
        first_day_price = price_series.iloc[j]
        while np.isnan(first_day_price) and j < len(price_series)-1:
            j = j+1
            bnh_series.append(0)
            total_capital.append(0)
            first_day_price = price_series.iloc[j]
        
        #print(j,len(price_series)-1)
        if j == len(price_series)-1:
          continue
                  
        # Buy at first day (j day)
        buy_pos = floor(balance/first_day_price)
        total_amt = buy_pos*first_day_price*(1+handle_fee)
        while balance < total_amt:
            buy_pos = buy_pos - 1
            total_amt = buy_pos*first_day_price*(1+handle_fee)
        balance = balance - total_amt
        bnh_series.append(balance+buy_pos*first_day_price) 
        total_capital.append(capital)
        
        # Hold during the year (from j+1 day to n-1 day)
        for k in range(j+1,len(price_series)-1):
            bnh_series.append(balance+buy_pos*price_series.iloc[k])
            total_capital.append(capital)
        
        # Sell at last day (n day)
        last_day_price = price_series.iloc[-1]
        total_amt = buy_pos*last_day_price*(1-handle_fee)
        buy_pos = 0
        balance = balance + total_amt
        bnh_series.append(balance)
        total_capital.append(capital)        
        
        summary["Buy&Hold_"+str(i+1)] = pd.Series(bnh_series,index = price_series.index)
        if i == 0:
            summary["Buy&Hold_Total"] = summary["Buy&Hold_"+str(i+1)]
            summary["Capital_Bal"] = pd.Series(total_capital,index = price_series.index)
            #print("First stock")
        else:
            summary["Buy&Hold_Total"] = summary["Buy&Hold_Total"] + summary["Buy&Hold_"+str(i+1)]
            summary["Capital_Bal"] = summary["Capital_Bal"] + pd.Series(total_capital,index = price_series.index)
            #print("other two stock")
        summary.drop(columns=["Buy&Hold_"+str(i+1)],inplace = True)
        
        #print(summary)
    summary["Portfolio_CAGR"]= ((summary["Total_Value"]-summary["Capital_Bal"])/summary["Capital_Bal"]*100).round(dp).astype(str)+"%"
    summary["Buy&Hold_CAGR"]= ((summary["Buy&Hold_Total"]-summary["Capital_Bal"])/summary["Capital_Bal"]*100).round(dp).astype(str)+"%"
    summary["Portfolio_Performance"] = ((summary["Total_Value"]-summary["Buy&Hold_Total"])/summary["Capital_Bal"]*100).round(dp).astype(str)+"%"
    
    return summary
    

# User Portfolio Backtesting

In [ ]:
year_list = [2019, 2020, 2021]
num_user = 1000
col_to_get_from_backtest = ["Action","Price","Pos_Bal","Cash_Bal","Cum_Profit","Total_Bal"]
col_to_fillna_in_backtest = ["Action","Price","Pos_Bal","Cash_Bal","Cum_Profit","Value","Sector","Stock"]

for year in year_list:
    skipped_list = []
    print("*****************")
    print("***** "+str(year)+" ******")
    print("*****************")
    user_portfo_summary = pd.DataFrame(columns=["UserID","Risk_Level","Year","Portfolio_Performance","Stock_Count"])
    for i in range(num_user):
        user_id = i+1 # Update the User ID

        # Create empty dataframe
        user_portfolio = pd.DataFrame()

        # Get top X from available model-stock-list
        top_X_list = get_user_sector_stock_by_id(user_df,volatility_summary,user_id,top = max_num_stock)
        #if user_id == -1:
        #print(top_X_list)
        
        if len(top_X_list)==0: # Skip that
            print("User", user_id, "is skipped")
            skipped_list.append(user_id)
            continue
        
        iter_num = len(top_X_list)

        # Get top X stock data
        for j in range(iter_num):
            # Get the stock name from top X list
            this_stock = (top_X_list.loc[j,"Model"],sector_sync(top_X_list.loc[j,"Sector"]),stock_sync(top_X_list.loc[j,"Stock"]))
            
            # Get the file path to read backtest
            filepath = list(filter(lambda f: (this_stock[0]+"_" in f) and ("_"+this_stock[1]+"-" in f) and ("-"+this_stock[2]+"_" in f) and (str(year) in f) ,os.listdir(record_folder)))
            #print(j,"--->",filepath)

            # Check if file exist
            try:
                assert(len(filepath)==1)
            except:
                iter_num = iter_num - 1
                print(filepath)
                print(this_stock)
                
            # Read the file
            this_record_df = pd.read_csv(record_folder + filepath[0],index_col = 0)
            get_data = this_record_df[col_to_get_from_backtest]
            get_data = get_data.rename(columns={"Total_Bal":"Value"})
            get_data["Model"] = this_stock[0]
            get_data["Sector"] = this_stock[1]
            get_data["Stock"] = this_stock[2]
            get_data = get_data.add_suffix("_"+str(j+1))
            if j == 0:
                user_portfolio = user_portfolio.append(get_data)
                #print(get_data)
            else:
                if len(get_data) > len(user_portfolio):
                    user_portfolio = user_portfolio.merge(get_data,how='right',left_index = True,right_index = True,suffixes=("", "_"))
                else:
                    user_portfolio = user_portfolio.merge(get_data,how='left',left_index = True,right_index = True,suffixes=("", "_"))

        for j in range(iter_num):
          user_portfolio[col_to_fillna_in_backtest[0]+"_"+str(j+1)].fillna(value = "Hold", inplace = True)
          for k in range(1,len(col_to_fillna_in_backtest)):
            user_portfolio[col_to_fillna_in_backtest[k]+"_"+str(j+1)].fillna(method ="ffill", inplace = True)

        user_portfolio["Total_Cash_Bal"] = user_portfolio[total_cash_col[:iter_num]].sum(axis=1)
        user_portfolio["Total_Value"] = user_portfolio[total_value_col[:iter_num]].sum(axis=1)
        user_portfolio["Total_Cum_Profit"] = user_portfolio[total_cum_profit[:iter_num]].sum(axis=1)
        #print(user_portfolio)
        user_portfolio = get_buy_and_hold_result(user_portfolio)
        user_portfolio.to_csv(user_test_folder+str(year)+"_test_user_"+str(user_id)+".csv")
        final_buyandhold_cagr = user_portfolio.loc[user_portfolio.index[-1],"Buy&Hold_CAGR"]
        final_portfolio_cagr = user_portfolio.loc[user_portfolio.index[-1],"Portfolio_CAGR"]
        final_portfolio_performance = user_portfolio.loc[user_portfolio.index[-1],"Portfolio_Performance"]

        risk_level = get_risk_level_by_id(user_df,user_id)
        user_portfo_summary = user_portfo_summary.append({"UserID":user_id,
                                                          "Risk_Level":risk_level,
                                                          "Year":year,
                                                          "Buy&Hold_CAGR":final_buyandhold_cagr,
                                                          "Portfolio_CAGR":final_portfolio_cagr,
                                                          "Portfolio_Performance":final_portfolio_performance,
                                                          "Stock_Count":iter_num},ignore_index = True)
        print("Finished Portfolio Backtesting in Year "+str(year)+" for User ID = "+str(user_id)+"\t| Risk Level: "+risk_level+"\t| Buy&Hold_CAGR: "+final_buyandhold_cagr+"   \t| Portfo_CAGR: "+final_portfolio_cagr+"\t| Portfo_Perf: "+final_portfolio_performance+"\t | Stock Count: "+str(iter_num))

        
    print("Finished all "+str(num_user)+" users in Year "+str(year)+"...")
    print("Skipped the following users:",skipped_list)
    user_portfo_summary.to_csv(volatility_folder+str(num_user)+"_users_in_"+str(year)+"_portfolio_summary.csv",index = False)
    print("Exported "+str(num_user)+" users summary in Year "+str(year)+"...")

print("**** Finsihed all ****")

*****************
***** 2019 ******
*****************
Finished Portfolio Backtesting in Year 2019 for User ID = 1	| Risk Level: low	| Buy&Hold_CAGR: 17.686%   	| Portfo_CAGR: 15.01%	| Portfo_Perf: -2.676%	 | Stock Count: 3
Finished Portfolio Backtesting in Year 2019 for User ID = 2	| Risk Level: medium	| Buy&Hold_CAGR: 37.224%   	| Portfo_CAGR: 45.546%	| Portfo_Perf: 8.323%	 | Stock Count: 3
Finished Portfolio Backtesting in Year 2019 for User ID = 3	| Risk Level: low	| Buy&Hold_CAGR: 27.072%   	| Portfo_CAGR: 27.395%	| Portfo_Perf: 0.323%	 | Stock Count: 3
Finished Portfolio Backtesting in Year 2019 for User ID = 4	| Risk Level: medium	| Buy&Hold_CAGR: 15.212%   	| Portfo_CAGR: 28.432%	| Portfo_Perf: 13.219%	 | Stock Count: 3
Finished Portfolio Backtesting in Year 2019 for User ID = 5	| Risk Level: medium	| Buy&Hold_CAGR: 15.212%   	| Portfo_CAGR: 28.432%	| Portfo_Perf: 13.219%	 | Stock Count: 3
Finished Portfolio Backtesting in Year 2019 for User ID = 6	| Risk Level: medium	| Buy&Hol

# Portfolio Backtest Statistics

In [ ]:
num_user = 1000
year_list = [2019, 2020, 2021]
levels = ["Low","Medium","High"]
attrs = ["Buy&Hold_CAGR","Portfolio_CAGR","Portfolio_Performance"]

for year in year_list:
  user_portfo_summary = pd.read_csv(volatility_folder+str(num_user)+"_users_in_"+str(year)+"_portfolio_summary.csv",index_col = None)
  for level in levels:
    msg = "====== "+level+" Level in "+str(year)+" ======"
    print("="*len(msg))
    print(msg)
    print("="*len(msg))
    count = len(user_portfo_summary[user_portfo_summary["Risk_Level"] == level.lower()])
    print("Count:",count,"\t| Percent:",str(round(count/num_user,4)*100)+"%")
    for attr in attrs:
      if attr != attrs[2]:
        print("*** Max ("+attr+"):",user_portfo_summary[user_portfo_summary["Risk_Level"] == level.lower()][attr].apply(lambda num: float(num[:-1])).max())
        print("*** Min ("+attr+"):",user_portfo_summary[user_portfo_summary["Risk_Level"] == level.lower()][attr].apply(lambda num: float(num[:-1])).min())
      print("*** Mean ("+attr+"):",user_portfo_summary[user_portfo_summary["Risk_Level"] == level.lower()][attr].apply(lambda num: float(num[:-1])).mean())
      print("* * * * * * * * * * * *")

====== Low Level in 2019 ======
Count: 139 	| Percent: 13.900000000000002%
*** Max (Buy&Hold_CAGR): 55.905
*** Min (Buy&Hold_CAGR): -20.036
*** Mean (Buy&Hold_CAGR): 25.083654676259
* * * * * * * * * * * *
*** Max (Portfolio_CAGR): 61.003
*** Min (Portfolio_CAGR): -20.842
*** Mean (Portfolio_CAGR): 26.668237410071963
* * * * * * * * * * * *
*** Mean (Portfolio_Performance): 1.5845755395683463
* * * * * * * * * * * *
====== Medium Level in 2019 ======
Count: 706 	| Percent: 70.6%
*** Max (Buy&Hold_CAGR): 91.813
*** Min (Buy&Hold_CAGR): -19.014
*** Mean (Buy&Hold_CAGR): 21.61472804532574
* * * * * * * * * * * *
*** Max (Portfolio_CAGR): 124.753
*** Min (Portfolio_CAGR): 4.597
*** Mean (Portfolio_CAGR): 38.81873937677064
* * * * * * * * * * * *
*** Mean (Portfolio_Performance): 17.20362039660044
* * * * * * * * * * * *
====== High Level in 2019 ======
Count: 148 	| Percent: 14.799999999999999%
*** Max (Buy&Hold_CAGR): 119.499
*** Min (Buy&Hold_CAGR): 2.287
*** Mean (Buy&Hold_CAGR): 77.950